In [99]:
import pandas as pd

In [100]:
import numpy as np


    header = None — загрузить без строки с заголовком;
    skiprows=n — пропустить n строк (часто у документов бывает техническая «шапка»);
    encoding — загрузить в конкретной кодировке;
    na_values — список значений, который нужно заменить на NaN (специальный объект, обозначающий пропущенное значение).


In [101]:
log = pd.read_csv("data/log.csv", header = None)

In [102]:
users = pd.read_csv("data/users.csv", encoding="KOI8-R", sep="	")

In [103]:
log.columns = ['user_id','time','bet','win']

In [104]:
users.columns = ['user_id','email','geo']

In [ ]:
log.head()

In [ ]:
log_win = log[log['win'] > 0]

In [ ]:
win_count = log_win.shape[0]
print(win_count)

In [ ]:
log2 = log.query("bet<2000 & win>0")

In [ ]:
log2

In [ ]:
log.user_id.unique()

In [105]:
new_log = log[~log.user_id.str.contains("error", na=False)]

In [106]:
def get_user_id(s):
    pos = s.find("user")
    if pos > -1:
        return s[pos:]
    else:
        return ""

In [107]:
log['user_id'] = log.user_id.apply(get_user_id)


In [108]:
#t = log.time[0]
log['time'] = log[log.time.str.match("\[", na=False)]['time'].apply(lambda t: str(t).replace('[', ''))
#t = t[1:]

In [113]:
log['time'] = pd.to_datetime(log['time'])

In [118]:
np.isnan(log.time.values).sum()

15

In [119]:
log_backup = log.copy()

In [129]:
log = log_backup.copy()

In [130]:
log.isnull().sum()

user_id      0
time        15
bet        515
win        862
dtype: int64

In [127]:
log = log.dropna(axis=0)

In [128]:
log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 14 to 991
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  133 non-null    object        
 1   time     133 non-null    datetime64[ns]
 2   bet      133 non-null    float64       
 3   win      133 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 5.2+ KB


In [125]:
log

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,user_984,2019-04-20 09:59:58,9754.0,NaN
996,,NaT,10054.0,29265.0
997,,NaT,10454.0,NaN
998,,NaT,1000.0,NaN
